In [1]:
import pandas as pd


df = pd.read_parquet("./all_cleaned_data/all_cleaned_data.parquet")
df.head()

,VendorID,PickupDatetime,DropoffDatetime,TripDuration,PassengerCount,TripDistance,PULocationID,DOLocationID,PaymentType,FareAmount,ExtraCharges,MTATax,TipAmount,TollsAmount,ImprovementSurcharge,TotalAmount,CongestionSurcharge,AirportFee
0,2,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.0,1.98,236,239,1.0,12.8,1.0,0.5,3.61,0.0,1.0,21.66,2.75,0.0
1,2,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,5.0,6.54,65,170,1.0,30.3,1.0,0.5,7.11,0.0,1.0,42.66,2.75,0.0
2,2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,1.0,3.08,74,262,1.0,19.8,1.0,0.5,3.00,0.0,1.0,28.05,2.75,0.0
3,1,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,1.0,2.40,74,116,2.0,14.2,1.0,1.5,0.00,0.0,1.0,16.70,0.00,0.0
4,2,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,1.0,5.14,74,243,1.0,22.6,1.0,0.5,6.28,0.0,1.0,31.38,0.00,0.0


In [2]:
# Need a function to filter data into what is essential for training
def necessary_fields(df : pd.DataFrame) -> pd.DataFrame: 
    return df.drop(columns=['VendorID',
                            'PassengerCount',
                            'PaymentType', 
                            'ExtraCharges', 
                            'MTATax', 
                            'TollsAmount', 
                            'ImprovementSurcharge', 
                            'TotalAmount', 
                            'CongestionSurcharge', 
                            'AirportFee']) # Keep Trip Distance and Duration as these may be helpful for the simulation.


df_fil = necessary_fields(df)
df_fil.head()

,PickupDatetime,DropoffDatetime,TripDuration,TripDistance,PULocationID,DOLocationID,FareAmount,TipAmount
0,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.98,236,239,12.8,3.61
1,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,6.54,65,170,30.3,7.11
2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,3.08,74,262,19.8,3.00
3,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,2.40,74,116,14.2,0.00
4,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,5.14,74,243,22.6,6.28


In [3]:
def merge_fare_and_tip(df: pd.DataFrame) -> pd.DataFrame:
    df_merged = df
    df_merged['TotalRevenue'] = df['FareAmount'] + df['TipAmount']
    return df_merged.drop(columns=['FareAmount', 'TipAmount'])

df_merged = merge_fare_and_tip(df_fil)
df_merged.head()

,PickupDatetime,DropoffDatetime,TripDuration,TripDistance,PULocationID,DOLocationID,TotalRevenue
0,2024-01-01 00:46:55,2024-01-01 00:58:25,11.500000,1.98,236,239,16.41
1,2024-01-01 00:31:42,2024-01-01 00:52:34,20.866667,6.54,65,170,37.41
2,2024-01-01 00:30:21,2024-01-01 00:49:23,19.033333,3.08,74,262,22.80
3,2024-01-01 00:30:20,2024-01-01 00:42:12,11.866667,2.40,74,116,14.20
4,2024-01-01 00:32:38,2024-01-01 00:43:37,10.983333,5.14,74,243,28.88


In [ ]:
from zone_coords import get_coords_as_dict
import math

lookup_coords = get_coords_as_dict()
print(lookup_coords)

def centroid_distances(PUZoneID, DOZoneID): # This is really just for use as a distance measure in the mean time until Oliver sorts out the actual shortest paths between zones. 
    lookup_coords = get_coords_as_dict()
    PUx = lookup_coords[PUZoneID]['x']
    PUy = lookup_coords[PUZoneID]['y']
    DOx = lookup_coords[DOZoneID]['x']
    DOy = lookup_coords[DOZoneID]['y']
    return math.sqrt((DOx - PUx)**2 + (DOy - PUy)**2)

print(centroid_distances(1,2))

{1: {'x': -74.17400027276298, 'y': 40.69183120640149}, 2: {'x': -73.83129854302214, 'y': 40.61674529165988}, 3: {'x': -73.84742223236718, 'y': 40.86447368477543}, 4: {'x': -73.97696825691767, 'y': 40.72375214158478}, 5: {'x': -74.18848410184931, 'y': 40.552659286945655}, 6: {'x': -74.07177055895137, 'y': 40.60032414603462}, 7: {'x': -73.91969431946073, 'y': 40.76149256216376}, 8: {'x': -73.9230861575075, 'y': 40.77855865375072}, 9: {'x': -73.7879488777789, 'y': 40.75103502557237}, 10: {'x': -73.79098635986959, 'y': 40.67895331011525}, 11: {'x': -74.00748815801178, 'y': 40.60427284003079}, 12: {'x': -74.01556351255896, 'y': 40.702945834984696}, 13: {'x': -74.01607915192938, 'y': 40.71203815707455}, 14: {'x': -74.0298923741879, 'y': 40.62483492360817}, 15: {'x': -73.78597227877711, 'y': 40.78333304726102}, 16: {'x': -73.77342181890141, 'y': 40.76273837544241}, 17: {'x': -73.9499050350751, 'y': 40.69150749488063}, 18: {'x': -73.89018328872034, 'y': 40.86768245607538}, 19: {'x': -73.726655

c:\Users\bendh\OneDrive\Documents\GitHub\adsproject\zone_coords.py:37: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df.T.to_dict()


In [ ]:
from datetime import datetime, timedelta
from dateutil import parser

def round_time_to_hour(tim):
    tim = parser.parse(tim)
    if tim.minute >= 30:
        return tim.replace(minute = 0, second = 0, microsecond=0) + timedelta(hours=1)
    else:
        return tim.replace(minute = 0, second = 0, microsecond=0)
    
def round_time_to_int(tim):
    tim = parser.parse(tim)
    if tim.minute >= 30:
        return int(tim.hour) + 1
    else:
        return int(tim.hour)

df_rounded = df_merged
df_rounded['PickupDatetimeRounded'] = df_merged['PickupDatetime'].map(lambda x: round_time_to_int(x))
df_rounded['DropoffDatetimeRounded'] = df_merged['DropoffDatetime'].map(lambda x: round_time_to_int(x))
df_rounded.head()

TypeError: Parser must be a string or character stream, not Series